In [3]:
# importa las bibliotecas necesarias y define la URL del video de YouTube que deseas descargar
import cv2
from pytube import YouTube

url = "https://www.youtube.com/watch?v=Vbl-pQL4v9M&list=PLv2tHoY9QR8peUi2cNXnXfSCI_w9jB8f4&index=2"


In [4]:
# Descarga el video de YouTube utilizando Pytube
youtube = YouTube(url)
video = youtube.streams.get_highest_resolution()
video.download()


'c:\\Users\\carlo\\OneDrive\\Documentos\\Extraccion de videos\\Presidente AMLO presenta ex funcionarios que frenaron desarrollo de CFE.mp4'

La siguiente celda es para poder seleccionar la region que nos interesa capturar y obtener las coordenadas de dicha region.

In [38]:
import cv2

# Seleccionar el video y capturar el primer cuadro
cap = cv2.VideoCapture('input.mp4')
ret, frame = cap.read()

# Mostrar el video y seleccionar la región deseada
region = cv2.selectROI(frame, fromCenter=False)

# Establecer el índice del cuadro inicial (en este caso, desde el segundo 30 del video)
cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_FPS) * 60)

x = 0
y = 0
w = 0
h = 0

# Utiliza el método "read" para obtener cada cuadro del video
while True:
    ret, frame = cap.read()

    # Si no se pueden obtener más cuadros, sal del bucle
    if not ret:
        break

    # Recortar la región deseada
    x, y, w, h = region
    frame = frame[y:y+h, x:x+w]

    # Escalar el cuadro si lo deseas
    #frame = cv2.resize(frame, (1280, 720))

    # Mostrar el cuadro y esperar para continuar
    cv2.imshow('Cuadro', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Liberar los recursos utilizados por OpenCV
cap.release()
cv2.destroyAllWindows()

# Guardar la región recortada en un archivo de imagen
cv2.imwrite('region_recortada.jpg', frame)



True

In [8]:
x, y, w, h

(979, 470, 257, 152)

In [36]:
import subprocess
import cv2
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.editor import *

video = cv2.VideoCapture('input.mp4')

# Definimos desde que segundo queremos que empiece y termine a capturar el video
inicio = 192
fin = 222

fps = video.get(cv2.CAP_PROP_FPS)

inicio_cuadro = int(inicio * video.get(cv2.CAP_PROP_FPS))
fin_cuadro = int(fin * video.get(cv2.CAP_PROP_FPS))

video.set(cv2.CAP_PROP_POS_FRAMES, inicio_cuadro)

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Coordenadas obtenidas gracias a la celda de arriba
x1, y1 = 977, 478 # coordenadas de la arista superior izquierda del cuadro de la region seleccionada
x2, y2 = 1208, 614 # coordenadas de la arista inferior derecha del cuadro de la region seleccionada

new_width, new_height = 854, 480 # dimensiones a las que se va escalar el video generado 854, 480

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('output.mp4', fourcc, fps, (new_width, new_height))



while True:
    ret, frame = video.read()

    if not ret:
        break

    cropped_frame = frame[y1:y2, x1:x2]

    # Convertir a escala de grises
    gray_frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)


    scaled_frame = cv2.resize(gray_frame, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

    # Convertir la imagen a color de nuevo 
    scaled_frame = cv2.cvtColor(scaled_frame, cv2.COLOR_GRAY2BGR)


    output_video.write(scaled_frame)

    if video.get(cv2.CAP_PROP_POS_FRAMES) == fin_cuadro:
        break

video.release()
output_video.release()


# Con opencv no podemos extraer el video con audio, por lo que hay que obtenerlo por separado y unirlo con el video que extraimos.

# Cargar el archivo de video original
video_clip = VideoFileClip('input.mp4')

# Recortar el video
recortado_clip = video_clip.subclip(inicio, fin)

# Extraer el audio del video recortado
audio = recortado_clip.audio
audio.write_audiofile('audio.mp3')

# Cargar el archivo de audio y de video
audio = AudioFileClip('audio.mp3')
video = VideoFileClip('output.mp4')

# Agregar el audio al video
final_clip = video.set_audio(audio)

# Escribir el video final con audio
final_clip.write_videofile('output_video.mp4', codec='libx264', fps=video.fps)






MoviePy - Writing audio in audio.mp3


MoviePy - Done.
Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
